In [ ]:
!pip install shap matplotlib

In [ ]:
!pip install scikit-learn

In [ ]:
# First, install required packages
!pip install streamlit
!pip install pyngrok
!pip install plotly
!pip install tensorflow



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.1 MB/s eta 0:00:00


In [ ]:
# Download ngrok (a tunneling service) and set it up
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2025-07-27 08:12:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 13.248.244.96, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  66.3MB/s    in 0.2s    

2025-07-27 08:12:33 (66.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# First, install required packages
!pip install streamlit
!pip install pyngrok
!pip install plotly
!pip install tensorflow


# Save your Streamlit app to a file
%%writefile app4.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
import warnings
warnings.filterwarnings('ignore')

# Page configuration
st.set_page_config(
    page_title="Advanced Sales Prediction AI",
    page_icon="🚀",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for modern look
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        font-weight: bold;
        background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        text-align: center;
        margin-bottom: 2rem;
    }
    .metric-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 1rem;
        border-radius: 10px;
        color: white;
        text-align: center;
        margin: 0.5rem 0;
    }
    .stButton > button {
        background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
        color: white;
        border: none;
        border-radius: 25px;
        padding: 0.5rem 2rem;
        font-weight: bold;
        transition: all 0.3s ease;
    }
    .stButton > button:hover {
        transform: translateY(-2px);
        box_shadow: 0 5px 15px rgba(0,0,0,0.3);
    }
    .upload-section {
        background: rgba(102, 126, 234, 0.1);
        padding: 2rem;
        border-radius: 15px;
        border: 2px dashed #667eea;
        text-align: center;
        margin: 2rem 0;
    }
    .nav-button {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 1rem 2rem;
        border-radius: 10px;
        text_decoration: none;
        display: inline-block;
        margin: 0.5rem;
        transition: all 0.3s ease;
    }
    .nav-button:hover {
        transform: translateY(-2px);
        box_shadow: 0 5px 15px rgba(0,0,0,0.3);
    }
    .reinforcement-card {
        background: linear_gradient(135deg, #ff6b6b 0%, #ee5a24 100%);
        padding: 1.5rem;
        border-radius: 15px;
        color: white;
        margin: 1rem 0;
    }
</style>
""", unsafe_allow_html=True)

# Initialize session state
if 'current_page' not in st.session_state:
    st.session_state.current_page = 'main'
if 'uploaded_data' not in st.session_state:
    st.session_state.uploaded_data = None
if 'models_trained' not in st.session_state:
    st.session_state.models_trained = False
if 'reinforcement_data' not in st.session_state:
    st.session_state.reinforcement_data = {}

@st.cache_data
def load_and_process_data(file):
    """Load and process the uploaded data with enhanced column detection"""
    try:
        if file.name.endswith('.csv'):
            df = pd.read_csv(file)
        else:
            df = pd.read_excel(file)

        # Clean column names
        df.columns = df.columns.str.strip()

        # Enhanced column detection for the new CSV format
        date_col = None
        product_col = None
        sales_per_unit_col = None
        quantity_col = None
        total_sales_col = None

        for col in df.columns:
            col_lower = col.lower()
            if 'date' in col_lower or 'order' in col_lower:
                date_col = col
            elif 'product' in col_lower or 'category' in col_lower:
                product_col = col
            elif 'sales per unit' in col_lower or 'price' in col_lower:
                sales_per_unit_col = col
            elif 'quantity' in col_lower or 'qty' in col_lower:
                quantity_col = col
            elif 'total sales' in col_lower or 'total' in col_lower:
                total_sales_col = col

        # Fallback column detection
        if date_col is None:
            for col in df.columns:
                if 'date' in col.lower():
                    date_col = col
                    break

        if product_col is None:
            for col in df.columns:
                if 'product' in col.lower() or 'category' in col.lower():
                    product_col = col
                    break

        if total_sales_col is None:
            for col in df.columns:
                if 'total' in col.lower():
                    total_sales_col = col
                    break

        if quantity_col is None:
            for col in df.columns:
                if 'quantity' in col.lower():
                    quantity_col = col
                    break

        if sales_per_unit_col is None:
            for col in df.columns:
                if 'sales per unit' in col.lower():
                    sales_per_unit_col = col
                    break

        # Validate required columns
        required_cols = [date_col, product_col, total_sales_col]
        if any(col is None for col in required_cols):
            st.error("Could not detect required columns. Please ensure your file has: Date, Product Category, and Total Sales columns.")
            st.error(f"Available columns: {list(df.columns)}")
            return None

        # Parse dates
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df = df.dropna(subset=[date_col])

        # Convert numeric columns
        if total_sales_col:
            df[total_sales_col] = pd.to_numeric(df[total_sales_col], errors='coerce')
        if quantity_col:
            df[quantity_col] = pd.to_numeric(df[quantity_col], errors='coerce')
        if sales_per_unit_col:
            df[sales_per_unit_col] = pd.to_numeric(df[sales_per_unit_col], errors='coerce')

        # Rename columns for consistency
        column_mapping = {
            date_col: 'date',
            product_col: 'product',
            total_sales_col: 'total_sales'
        }

        if quantity_col:
            column_mapping[quantity_col] = 'quantity'
        if sales_per_unit_col:
            column_mapping[sales_per_unit_col] = 'sales_per_unit'

        df = df.rename(columns=column_mapping)

        # Sort by date
        df = df.sort_values('date')

        return df

    except Exception as e:
        st.error(f"Error loading file: {str(e)}")
        return None

def create_advanced_regression_models():
    """Create multiple regression models for ensemble prediction"""
    models = {
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Lasso Regression': Lasso(alpha=0.1),
        'SVR': SVR(kernel='rbf', C=100, gamma='scale')
    }
    return models

def create_lstm_model(input_shape):
    """Create advanced LSTM model for time series prediction"""
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        Dropout(0.3),
        LSTM(64, return_sequences=True),
        Dropout(0.3),
        LSTM(32, return_sequences=False),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

def prepare_features_for_regression(df, product):
    """Prepare features for regression models"""
    product_data = df[df['product'] == product].copy()

    if len(product_data) < 10:
        return None, None, None

    # Create time-based features
    product_data['year'] = product_data['date'].dt.year
    product_data['month'] = product_data['date'].dt.month
    product_data['day'] = product_data['date'].dt.day
    product_data['day_of_week'] = product_data['date'].dt.dayofweek
    product_data['quarter'] = product_data['date'].dt.quarter

    # Create lag features
    product_data = product_data.sort_values('date')
    product_data['sales_lag_1'] = product_data['total_sales'].shift(1)
    product_data['sales_lag_2'] = product_data['total_sales'].shift(2)
    product_data['sales_lag_3'] = product_data['total_sales'].shift(3)

    # Create rolling features
    product_data['sales_rolling_mean_3'] = product_data['total_sales'].rolling(window=3).mean()
    product_data['sales_rolling_mean_7'] = product_data['total_sales'].rolling(window=7).mean()
    product_data['sales_rolling_std_3'] = product_data['total_sales'].rolling(window=3).std()

    # Drop NaN values
    product_data = product_data.dropna()

    if len(product_data) < 5:
        return None, None, None

    # Prepare features
    feature_columns = ['year', 'month', 'day', 'day_of_week', 'quarter',
                      'sales_lag_1', 'sales_lag_2', 'sales_lag_3',
                      'sales_rolling_mean_3', 'sales_rolling_mean_7', 'sales_rolling_std_3']

    # Add quantity and sales_per_unit if available
    if 'quantity' in product_data.columns:
        feature_columns.append('quantity')
    if 'sales_per_unit' in product_data.columns:
        feature_columns.append('sales_per_unit')

    X = product_data[feature_columns]
    y = product_data['total_sales']

    return X, y, product_data

def train_ensemble_models(X, y, product):
    """Train multiple regression models and return ensemble predictions"""
    models = create_advanced_regression_models()
    trained_models = {}
    predictions = {}
    scores = {}

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    for name, model in models.items():
        try:
            # Train model
            model.fit(X_train_scaled, y_train)

            # Make predictions
            y_pred = model.predict(X_test_scaled)

            # Calculate metrics
            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            trained_models[name] = model
            predictions[name] = y_pred
            scores[name] = {
                'MSE': mse,
                'MAE': mae,
                'R2': r2,
                'RMSE': np.sqrt(mse)
            }

        except Exception as e:
            st.warning(f"Error training {name} for {product}: {str(e)}")
            continue

    return trained_models, predictions, scores, scaler

def predict_future_regression(trained_models, scaler, last_data, steps=6):
    """Predict future values using ensemble models"""
    predictions = {}

    for name, model in trained_models.items():
        try:
            future_predictions = []
            current_data = last_data.copy()

            for _ in range(steps):
                # Prepare features for next prediction
                features = current_data.reshape(1, -1)
                features_scaled = scaler.transform(features)

                # Predict next value
                next_pred = model.predict(features_scaled)[0]
                future_predictions.append(next_pred)

                # Update features for next iteration (simplified)
                # In a real scenario, you'd need to update all time-based features
                current_data[0] = next_pred  # Update the target variable

            predictions[name] = future_predictions

        except Exception as e:
            st.warning(f"Error predicting with {name}: {str(e)}")
            continue

    return predictions

def create_reinforcement_learning_agent():
    """Create a simple reinforcement learning agent for business optimization"""
    class BusinessRLAgent:
        def __init__(self, n_actions=5):
            self.n_actions = n_actions
            self.q_table = {}
            self.learning_rate = 0.1
            self.discount_factor = 0.95
            self.epsilon = 0.1

        def get_state_key(self, state):
            return str(state)

        def get_action(self, state):
            state_key = self.get_state_key(state)

            if state_key not in self.q_table:
                self.q_table[state_key] = np.zeros(self.n_actions)

            # Epsilon-greedy strategy
            if np.random.random() < self.epsilon:
                return np.random.randint(self.n_actions)
            else:
                return np.argmax(self.q_table[state_key])

        def update(self, state, action, reward, next_state):
            state_key = self.get_state_key(state)
            next_state_key = self.get_state_key(next_state)

            if state_key not in self.q_table:
                self.q_table[state_key] = np.zeros(self.n_actions)
            if next_state_key not in self.q_table:
                self.q_table[next_state_key] = np.zeros(self.n_actions)

            # Q-learning update
            current_q = self.q_table[state_key][action]
            max_next_q = np.max(self.q_table[next_state_key])
            new_q = current_q + self.learning_rate * (reward + self.discount_factor * max_next_q - current_q)
            self.q_table[state_key][action] = new_q

            def sidebar_nav(df: pd.DataFrame | None):
              with st.sidebar:
                pass # Placeholder for sidebar navigation

    return BusinessRLAgent()

def generate_business_recommendations(df, product_models, predictions):
    """Generate business recommendations using reinforcement learning"""
    recommendations = {}

    for product in df['product'].unique():
        if product in product_models:
            product_data = df[df['product'] == product]

            # Calculate business metrics
            avg_sales = product_data['total_sales'].mean()
            sales_volatility = product_data['total_sales'].std()
            total_revenue = product_data['total_sales'].sum()

            # Create RL agent
            agent = create_reinforcement_learning_agent()

            # Simulate business scenarios
            scenarios = []
            for _ in range(100):
                # Define state based on current metrics
                state = [
                    int(avg_sales > product_data['total_sales'].median()),
                    int(sales_volatility > product_data['total_sales'].std()),
                    int(total_revenue > product_data['total_sales'].sum() * 0.8)
                ]

                action = agent.get_action(state)

                # Calculate reward based on predicted performance
                if product in predictions:
                    predicted_avg = np.mean(predictions[product]['ensemble'])
                    reward = 1 if predicted_avg > avg_sales else -0.5
                else:
                    reward = 0

                # Update agent
                next_state = state.copy()
                agent.update(state, action, reward, next_state)

                scenarios.append({
                    'action': action,
                    'reward': reward,
                    'state': state
                })

            # Generate recommendations based on best actions
            best_action = np.argmax([s['reward'] for s in scenarios])

            action_descriptions = {
                0: "Increase marketing budget",
                1: "Optimize pricing strategy",
                2: "Improve inventory management",
                3: "Enhance customer service",
                4: "Expand product line"
            }

            recommendations[product] = {
                'recommended_action': action_descriptions[best_action],
                'confidence_score': np.mean([s['reward'] for s in scenarios]),
                'current_avg_sales': avg_sales,
                'predicted_improvement': predicted_avg - avg_sales if product in predictions else 0,
                'risk_level': 'High' if sales_volatility > avg_sales * 0.5 else 'Low'
            }

    return recommendations

def main_page():
    """Main dashboard page"""
    st.markdown('<h1 class="main-header">🚀 Advanced Sales Prediction AI</h1>', unsafe_allow_html=True)
    st.markdown("### Transform your transaction data into actionable sales insights with Advanced AI & Reinforcement Learning")

    # Navigation buttons
    col1, col2, col3 = st.columns(3)

    with col1:
        if st.button("📊 Historical Sales Prediction", type="primary", use_container_width=True):
            st.session_state.current_page = 'historical'
            st.rerun()

    with col2:
        if st.button("🤖 Reinforcement Learning", type="primary", use_container_width=True):
            st.session_state.current_page = 'reinforcement'
            st.rerun()

    with col3:
        if st.button("📈 Business Analytics", type="primary", use_container_width=True):
            st.session_state.current_page = 'analytics'
            st.rerun()

    # File upload section
    st.markdown('<div class="upload-section">', unsafe_allow_html=True)
    st.markdown("### 📁 Upload Your Transaction Data")
    uploaded_file = st.file_uploader(
        "Choose a CSV or Excel file",
        type=['csv', 'xlsx', 'xls'],
        help="Upload your transaction data with columns: Order Date, Product Category, Sales per Unit, Quantity, Total Sales"
    )
    st.markdown('</div>', unsafe_allow_html=True)

    if uploaded_file is not None:
        with st.spinner("🔄 Loading and processing your data..."):
            df = load_and_process_data(uploaded_file)
            st.session_state.uploaded_data = df

        if df is not None:
            st.success(f"✅ Data loaded successfully! Found {len(df)} transactions across {df['product'].nunique()} products")

            # Store data in session state
            st.session_state.uploaded_data = df

            # Data overview
            col1, col2, col3, col4 = st.columns(4)

            with col1:
                st.metric("Total Transactions", len(df))

            with col2:
                st.metric("Unique Products", df['product'].nunique())

            with col3:
                st.metric("Date Range", f"{df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")

            with col4:
                st.metric("Total Revenue", f"${df['total_sales'].sum():,.0f}")

def historical_prediction_page():
    """Historical sales prediction page"""
    st.markdown('<h1 class="main-header">📊 Historical Sales Prediction</h1>', unsafe_allow_html=True)

    # Back button
    if st.button("← Back to Main Menu"):
        st.session_state.current_page = 'main'
        st.rerun()

    if st.session_state.uploaded_data is None:
        st.warning("Please upload data from the main menu first.")
        return

    df = st.session_state.uploaded_data

    # Product selection
    st.markdown("## 🎯 Select Products for Advanced Analysis")
    products = df['product'].unique()
    selected_products = st.multiselect(
        "Choose products to analyze",
        products,
        default=products[:min(5, len(products))],
        help="Select products for advanced regression analysis and prediction"
    )

    if selected_products:
        # Prediction period selection
        col1, col2 = st.columns(2)
        with col1:
            prediction_period = st.selectbox(
                "Select Prediction Period",
                ["1 Month", "3 Months", "6 Months", "1 Year"],
                index=2
            )

        with col2:
            show_ensemble = st.checkbox(
                "Show Ensemble Model Comparison",
                value=True
            )

        if st.button("🚀 Train Advanced AI Models", type="primary"):
            progress_bar = st.progress(0)
            status_text = st.empty()

            product_models = {}
            all_predictions = {}
            all_scores = {}

            for i, product in enumerate(selected_products):
                status_text.text(f"Training advanced models for {product}...")
                progress_bar.progress((i + 1) / len(selected_products))

                # Prepare features
                X, y, product_data = prepare_features_for_regression(df, product)

                if X is not None:
                    # Train ensemble models
                    trained_models, predictions, scores, scaler = train_ensemble_models(X, y, product)

                    if trained_models:
                        product_models[product] = {
                            'models': trained_models,
                            'scaler': scaler,
                            'data': product_data
                        }

                        all_scores[product] = scores

                        # Predict future values
                        last_data = X.iloc[-1].values
                        future_predictions = predict_future_regression(trained_models, scaler, last_data, steps=6)

                        # Calculate ensemble prediction
                        if future_predictions:
                            ensemble_predictions = []
                            for step in range(6):
                                step_predictions = [pred[step] for pred in future_predictions.values() if len(pred) > step]
                                if step_predictions:
                                    ensemble_predictions.append(np.mean(step_predictions))

                            all_predictions[product] = {
                                'individual': future_predictions,
                                'ensemble': ensemble_predictions
                            }

            progress_bar.progress(1.0)
            status_text.text("✅ Advanced AI models trained successfully!")

            # Store results in session state
            st.session_state.product_models = product_models
            st.session_state.all_predictions = all_predictions
            st.session_state.all_scores = all_scores
            st.session_state.models_trained = True

            # Display results
            display_prediction_results(all_predictions, all_scores, show_ensemble)

def display_prediction_results(predictions, scores, show_ensemble):
    """Display prediction results with advanced visualizations"""
    if not predictions:
        st.warning("No predictions available. Please check your data and try again.")
        return

    st.markdown("## 📈 Advanced Prediction Results")

    # Create tabs for different views
    tab1, tab2, tab3 = st.tabs(["🎯 Predictions", "📊 Model Performance", "📈 Ensemble Analysis"])

    with tab1:
        st.markdown("### Future Sales Predictions")

        # Create prediction visualization
        fig = go.Figure()

        for product, pred_data in predictions.items():
            if 'ensemble' in pred_data:
                # Historical data
                historical_data = st.session_state.product_models[product]['data']

                fig.add_trace(go.Scatter(
                    x=historical_data['date'],
                    y=historical_data['total_sales'],
                    mode='lines+markers',
                    name=f"{product} (Historical)",
                    line=dict(width=2)
                ))

                # Ensemble predictions
                future_dates = pd.date_range(
                    start=historical_data['date'].iloc[-1] + pd.DateOffset(days=1),
                    periods=len(pred_data['ensemble']),
                    freq='M'
                )

                fig.add_trace(go.Scatter(
                    x=future_dates,
                    y=pred_data['ensemble'],
                    mode='lines+markers',
                    name=f"{product} (Predicted)",
                    line=dict(width=2, dash='dash'),
                    marker=dict(symbol='diamond')
                ))

        fig.update_layout(
            title="Historical Sales vs AI Predictions",
            xaxis_title="Date",
            yaxis_title="Sales Amount",
            height=600,
            showlegend=True
        )

        st.plotly_chart(fig, use_container_width=True)

    with tab2:
        st.markdown("### Model Performance Comparison")

        if show_ensemble and scores:
            # Create performance comparison table
            performance_data = []

            for product, product_scores in scores.items():
                for model_name, metrics in product_scores.items():
                    performance_data.append({
                        'Product': product,
                        'Model': model_name,
                        'R² Score': f"{metrics['R2']:.3f}",
                        'RMSE': f"${metrics['RMSE']:.2f}",
                        'MAE': f"${metrics['MAE']:.2f}"
                    })

            performance_df = pd.DataFrame(performance_data)
            st.dataframe(performance_df, use_container_width=True)

            # Performance heatmap
            pivot_df = performance_df.pivot(index='Product', columns='Model', values='R² Score')
            pivot_df = pivot_df.apply(pd.to_numeric, errors='coerce')

            fig = px.imshow(
                pivot_df,
                title="Model Performance Heatmap (R² Scores)",
                color_continuous_scale='RdYlGn'
            )
            st.plotly_chart(fig, use_container_width=True)

    with tab3:
        st.markdown("### Ensemble Model Analysis")

        if predictions:
            # Compare individual models vs ensemble
            fig = go.Figure()

            for product, pred_data in predictions.items():
                if 'individual' in pred_data:
                    # Plot individual model predictions
                    for model_name, model_preds in pred_data['individual'].items():
                        if len(model_preds) > 0:
                            fig.add_trace(go.Scatter(
                                x=list(range(len(model_preds))),
                                y=model_preds,
                                mode='lines+markers',
                                name=f"{product} - {model_name}",
                                line=dict(width=1, dash='dot')
                            ))

                    # Plot ensemble prediction
                    if 'ensemble' in pred_data:
                        fig.add_trace(go.Scatter(
                            x=list(range(len(pred_data['ensemble']))),
                            y=pred_data['ensemble'],
                            mode='lines+markers',
                            name=f"{product} - Ensemble",
                            line=dict(width=3),
                            marker=dict(symbol='diamond', size=8)
                        ))

            fig.update_layout(
                title="Individual Models vs Ensemble Predictions",
                xaxis_title="Prediction Steps",
                yaxis_title="Predicted Sales",
                height=500,
                showlegend=True
            )

            st.plotly_chart(fig, use_container_width=True)

def reinforcement_learning_page():
    """Reinforcement learning for business optimization"""
    st.markdown('<h1 class="main-header">🤖 Reinforcement Learning Business Optimization</h1>', unsafe_allow_html=True)

    # Back button
    if st.button("← Back to Main Menu"):
        st.session_state.current_page = 'main'
        st.rerun()

    if st.session_state.uploaded_data is None:
        st.warning("Please upload data from the main menu first.")
        return

    df = st.session_state.uploaded_data

    st.markdown("## 🎯 Business Optimization with AI")
    st.markdown("This page uses reinforcement learning to provide business recommendations for improving sales performance.")

    # Product selection for RL analysis
    products = df['product'].unique()
    selected_products = st.multiselect(
        "Choose products for business optimization",
        products,
        default=products[:min(3, len(products))],
        help="Select products to analyze with reinforcement learning"
    )

    if selected_products and st.button("🚀 Generate Business Recommendations", type="primary"):
        with st.spinner("🤖 Training reinforcement learning agent..."):
            # Generate recommendations
            recommendations = generate_business_recommendations(df,
                                                           st.session_state.get('product_models', {}),
                                                           st.session_state.get('all_predictions', {}))

            st.session_state.reinforcement_data = recommendations

            # Display recommendations
            display_business_recommendations(recommendations, selected_products)

def display_business_recommendations(recommendations, selected_products):
    """Display business recommendations with visualizations"""
    st.markdown("## 💡 AI Business Recommendations")

    # Create tabs for different recommendation views
    tab1, tab2, tab3 = st.tabs(["🎯 Recommendations", "📊 Performance Analysis", "🚀 Action Plan"])

    with tab1:
        st.markdown("### Product-Specific Recommendations")

        for product in selected_products:
            if product in recommendations:
                rec = recommendations[product]

                with st.container():
                    st.markdown(f"### {product}")

                    col1, col2, col3 = st.columns(3)

                    with col1:
                        st.metric("Current Avg Sales", f"${rec['current_avg_sales']:.2f}")

                    with col2:
                        st.metric("Predicted Improvement", f"${rec['predicted_improvement']:.2f}")

                    with col3:
                        st.metric("Risk Level", rec['risk_level'])

                    # Recommendation card
                    st.markdown(f"""
                    <div class="reinforcement-card">
                        <h4>🎯 Recommended Action</h4>
                        <p><strong>{rec['recommended_action']}</strong></p>
                        <p>Confidence Score: {rec['confidence_score']:.2f}</p>
                    </div>
                    """, unsafe_allow_html=True)

    with tab2:
        st.markdown("### Performance Analysis")

        if recommendations:
            # Create performance comparison chart
            products = list(recommendations.keys())
            current_sales = [recommendations[p]['current_avg_sales'] for p in products]
            predicted_improvements = [recommendations[p]['predicted_improvement'] for p in products]
            confidence_scores = [recommendations[p]['confidence_score'] for p in products]

            fig = go.Figure()

            fig.add_trace(go.Bar(
                x=products,
                y=current_sales,
                name='Current Avg Sales',
                marker_color='lightblue'
            ))

            fig.add_trace(go.Bar(
                x=products,
                y=predicted_improvements,
                name='Predicted Improvement',
                marker_color='lightgreen'
            ))

            fig.update_layout(
                title="Current vs Predicted Performance",
                xaxis_title="Products",
                yaxis_title="Sales Amount ($)",
                barmode='group',
                height=500
            )

            st.plotly_chart(fig, use_container_width=True)

            # Confidence scores
            fig2 = px.bar(
                x=products,
                y=confidence_scores,
                title="AI Confidence Scores",
                labels={'x': 'Products', 'y': 'Confidence Score'}
            )

            st.plotly_chart(fig2, use_container_width=True)

    with tab3:
        st.markdown("### 🚀 Action Plan")

        if recommendations:
            st.markdown("### Prioritized Action Items")

            # Sort recommendations by confidence score
            sorted_recs = sorted(recommendations.items(),
                               key=lambda x: x[1]['confidence_score'],
                               reverse=True)

            for i, (product, rec) in enumerate(sorted_recs, 1):
                st.markdown(f"""
                **{i}. {product}**
                - **Action**: {rec['recommended_action']}
                - **Expected Impact**: ${rec['predicted_improvement']:.2f} improvement
                - **Confidence**: {rec['confidence_score']:.2f}
                - **Risk Level**: {rec['risk_level']}
                """)

            # Summary metrics
            st.markdown("### 📊 Summary")

            col1, col2, col3 = st.columns(3)

            with col1:
                total_improvement = sum(rec['predicted_improvement'] for rec in recommendations.values())
                st.metric("Total Predicted Improvement", f"${total_improvement:.2f}")

            with col2:
                avg_confidence = np.mean([rec['confidence_score'] for rec in recommendations.values()])
                st.metric("Average Confidence", f"{avg_confidence:.2f}")

            with col3:
                high_risk_count = sum(1 for rec in recommendations.values() if rec['risk_level'] == 'High')
                st.metric("High Risk Products", high_risk_count)

def analytics_page():
    """Business analytics page"""
    st.markdown('<h1 class="main-header">📈 Business Analytics Dashboard</h1>', unsafe_allow_html=True)

    # Back button
    if st.button("← Back to Main Menu"):
        st.session_state.current_page = 'main'
        st.rerun()

    if st.session_state.uploaded_data is None:
        st.warning("Please upload data from the main menu first.")
        return

    df = st.session_state.uploaded_data

    st.markdown("## 📊 Advanced Business Analytics")

    # Create tabs for different analytics
    tab1, tab2, tab3 = st.tabs(["📈 Sales Trends", "🎯 Product Analysis", "📊 Revenue Insights"])

    with tab1:
        st.markdown("### Sales Trends Analysis")

        # Monthly sales trend
        monthly_sales = df.groupby(df['date'].dt.to_period('M'))['total_sales'].sum()

        fig = px.line(
            x=monthly_sales.index.astype(str),
            y=monthly_sales.values,
            title="Monthly Sales Trend",
            labels={'x': 'Month', 'y': 'Total Sales'}
        )
        st.plotly_chart(fig, use_container_width=True)

        # Product performance over time
        product_monthly = df.groupby([df['date'].dt.to_period('M'), 'product'])['total_sales'].sum().reset_index()
        product_monthly['date'] = product_monthly['date'].astype(str)

        fig2 = px.line(
            product_monthly,
            x='date',
            y='total_sales',
            color='product',
            title="Product Sales Trends"
        )
        st.plotly_chart(fig2, use_container_width=True)

    with tab2:
        st.markdown("### Product Performance Analysis")

        # Top products by revenue
        top_products = df.groupby('product')['total_sales'].sum().sort_values(ascending=False)

        fig = px.bar(
            x=top_products.values,
            y=top_products.index,
            orientation='h',
            title="Top Products by Revenue"
        )
        st.plotly_chart(fig, use_container_width=True)

        # Product sales distribution
        fig2 = px.box(
            df,
            x='product',
            y='total_sales',
            title="Sales Distribution by Product"
        )
        st.plotly_chart(fig2, use_container_width=True)

    with tab3:
        st.markdown("### Revenue Insights")

        # Revenue metrics
        col1, col2, col3, col4 = st.columns(4)

        with col1:
            total_revenue = df['total_sales'].sum()
            st.metric("Total Revenue", f"${total_revenue:,.0f}")

        with col2:
            avg_order_value = df['total_sales'].mean()
            st.metric("Average Order Value", f"${avg_order_value:.2f}")

        with col3:
            total_orders = len(df)
            st.metric("Total Orders", total_orders)

        with col4:
            unique_customers = df['product'].nunique()  # Assuming product as proxy for customer segments
            st.metric("Product Categories", unique_customers)

        # Revenue by product category
        revenue_by_product = df.groupby('product')['total_sales'].agg(['sum', 'mean', 'count']).reset_index()
        revenue_by_product.columns = ['Product', 'Total Revenue', 'Average Order Value', 'Number of Orders']

        st.dataframe(revenue_by_product, use_container_width=True)

# Main application logic
def main():
    # Page routing
    if st.session_state.current_page == 'main':
        main_page()
    elif st.session_state.current_page == 'historical':
        historical_prediction_page()
    elif st.session_state.current_page == 'reinforcement':
        reinforcement_learning_page()
    elif st.session_state.current_page == 'analytics':
        analytics_page()

if __name__ == "__main__":
    main()



# [Rest of your Streamlit code goes here...]
# Paste ALL your Streamlit code from your original file here

Writing app4.py


In [ ]:
# The code below will run the Streamlit app and create a public URL
!streamlit run app.py &>/dev/null&
!./ngrok http 8501

# Get the public URL
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url']"

Usage of ngrok requires a verified account and authtoken.

Sign up for an account: https://dashboard.ngrok.com/signup
Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken

ERR_NGROK_4018

  File "<string>", line 1
    import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url']
                           ^
SyntaxError: '(' was never closed


In [ ]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 2s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.31.94.136


In [ ]:
!streamlit run app8.py & npx localtunnel --port 8501

⠙⠹

⠸⠼⠴⠦⠧⠇⠏⠋⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.31.94.136:8501

⠹⠸⠼⠴⠦⠧⠇⠏your url is: https://clear-paws-help.loca.lt
2025-08-14 09:33:19.756958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755163999.787870   25181 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755163999.798218   25181 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755163999.822756   25181 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755163999.